In [7]:
from sqlalchemy import create_engine
import cx_Oracle
import pandas as pd
from sklearn.metrics import classification_report

from sklearn.ensemble import BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn import model_selection

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#importing all algorithms in sklearn
from sklearn.svm import SVC

#linear classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis #quadratic instead of line
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn import neighbors #knn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import datasets
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing


import xgboost as xgb
import numpy as np

host='192.168.113.80'
port=1521
sid='bbidb'
user='system'
password='oraclebbi'
sid = cx_Oracle.makedsn(host, port, sid=sid)


In [8]:
def oracle_connect(host=host, sid=sid, user=user, password=password):
    cstr = 'oracle://{user}:{password}@{sid}/'.format (
        user = user ,
        password = password ,
        sid = sid
    )

    engine = create_engine (
        cstr ,
        convert_unicode = False ,
        pool_recycle = 10 ,
        pool_size = 50 ,
        echo = True
    )

    return engine.connect()

In [9]:
con=oracle_connect(host, sid, user, password)

2019-10-21 11:37:38,967 INFO sqlalchemy.engine.base.Engine SELECT USER FROM DUAL
2019-10-21 11:37:38,970 INFO sqlalchemy.engine.base.Engine {}
2019-10-21 11:37:38,976 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60 CHAR)) AS anon_1 FROM DUAL
2019-10-21 11:37:38,977 INFO sqlalchemy.engine.base.Engine {}
2019-10-21 11:37:38,981 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR2(60 CHAR)) AS anon_1 FROM DUAL
2019-10-21 11:37:38,982 INFO sqlalchemy.engine.base.Engine {}
2019-10-21 11:37:38,984 INFO sqlalchemy.engine.base.Engine SELECT CAST('test nvarchar2 returns' AS NVARCHAR2(60)) AS anon_1 FROM DUAL
2019-10-21 11:37:38,985 INFO sqlalchemy.engine.base.Engine {}
2019-10-21 11:37:38,989 INFO sqlalchemy.engine.base.Engine SELECT value FROM v$parameter WHERE name = 'compatible'
2019-10-21 11:37:38,990 INFO sqlalchemy.engine.base.Engine {}
2019-10-21 11:37:39,002 INFO sqlalchemy.engine.base.Engine select value from nls_session_pa

In [10]:
data=pd.read_sql_query(sql='select * from AI_LAGORTA.CAMP_AUG15_Y_CBM2', con=con) 

2019-10-21 11:37:39,577 INFO sqlalchemy.engine.base.Engine select * from AI_LAGORTA.CAMP_AUG15_Y_CBM2
2019-10-21 11:37:39,578 INFO sqlalchemy.engine.base.Engine {}


In [11]:
x=data.copy()
del x['pk_msisdn_id']
data['target_all'].value_counts()

0    22967
1      851
Name: target_all, dtype: int64

In [12]:
y = x[['target_all']].values
del x['target_all']

In [13]:
enc=preprocessing.LabelEncoder()
x=x.apply(enc.fit_transform)
y=enc.fit_transform(y)
x=np.array(x)
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size = 0.3, stratify = y)
x_train

C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[0, 0, 0, ..., 1, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 0, 1],
       ...,
       [0, 0, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 0, 1]])

# xgboosting with actual data (no ovrer sampling here at all)

In [14]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(x_train, y_train)
print("xgb ",gbm.score(x_test,y_test))

y_score =  gbm.predict(x_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_score, target_names=target_names))

xgb  0.9643157010915198
              precision    recall  f1-score   support

     class 0       0.96      1.00      0.98      6891
     class 1       0.00      0.00      0.00       255

    accuracy                           0.96      7146
   macro avg       0.48      0.50      0.49      7146
weighted avg       0.93      0.96      0.95      7146



C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# over sample the data

In [15]:
#over sample the data
from imblearn.over_sampling import SMOTE
from collections import Counter
sm = SMOTE()
X_res, y_res = sm.fit_resample(x_train, y_train)

print(Counter(y_train))
print(Counter(y_res))

C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint

Counter({0: 16076, 1: 596})
Counter({0: 16076, 1: 16076})


# xgboosting with over sampled data

In [16]:
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_res, y_res)

y_score2 =  gbm.predict(x_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_test, y_score2, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.97      0.89      0.93      6891
     class 1       0.06      0.18      0.09       255

    accuracy                           0.86      7146
   macro avg       0.51      0.54      0.51      7146
weighted avg       0.93      0.86      0.90      7146



In [17]:
y_scor =  gbm.predict(X_res)
target_names = ['class 0', 'class 1']
print(classification_report(y_res, y_scor, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.89      0.87     16076
     class 1       0.89      0.84      0.86     16076

    accuracy                           0.87     32152
   macro avg       0.87      0.87      0.87     32152
weighted avg       0.87      0.87      0.87     32152

